# Problem 2

In [1]:
import numpy as np
import pandas as pd
import multiprocess
### PLEASE use pip install multiprocess
from haversine import haversine as hv
from dateutil.parser import parse as pardat
import folium 
from folium.plugins import HeatMap
from geopy.distance import great_circle as GRC
import time 

In [2]:
data = pd.read_csv('combined_trajectories.csv',usecols=['latitude','longitude','time','individual_id','trajectory_id'])#,nrows =100000)
print('Before cleaning:', len(data),'rows')

## There were some faulty out of range longitude-latitude values in the data. We're dropping all such rows 
data  = data.loc[  (data['latitude'] >= -90)  & (data['latitude'] <= 90) & (data['longitude'] >= -180)&(data['longitude'] <= 180)]
data.reset_index(drop = True, inplace = True)

print('After cleaning: ', len(data),'rows')

# Group the data by `individual_id` to get distance of each individual 
grp_per = data.groupby(['individual_id'],sort= False)

Before cleaning: 24876978 rows
After cleaning:  24876977 rows



# Part 1
#### compute_dist(): function to calculate distance travelled by `one` individual

In [ ]:
def get_dist(row):
    # uses the haversines() function 
    return hv((row['latitude'],row['longitude']), (row['p_lat'],row['p_lng']))

def compute_dist(tup):
    (id ,trip_data) = tup
    trip_data.reset_index(drop = True, inplace = True)

    # Created 'previous_latitude` and `previous_longitude` columns shifted from trip_dist by 1
    x = trip_data.loc[1:,['latitude','longitude']]
    x.reset_index(drop = True, inplace = True)

    # Concatenate with trip_dist
    trip_data[['p_lat','p_lng']] =  x
    trip_data.drop(index=trip_data.index[-1],inplace = True )
    trip_data.reset_index(drop = True, inplace = True)


    # Use vectorisation, .apply() function
    return trip_data.apply(lambda row: hv((row['latitude'],row['longitude']),(row['p_lat'], row['p_lng'])) ,axis=1).sum()
 
# NOTE: While this vectorisation method rus in jut 3 minutes, the nive method using loops takes over 56 minutes to run


### Running the funciton to compute distances by `Multiprocessing` 

In [23]:
st = time.time()
if __name__ == '__main__':
    # Create a pool() of precesses 
    pool = multiprocess.Pool()

    # Each individual_id corresponds to a process(). Start then all parallely
    travelled  = pool.map(compute_dist, grp_per)
    pool.close()
    pool.join()
    
    en = time.time()
    print('Runtime to compute dustance traveled by each persion: ',en-st , 'seconds')
    print('Distance travelled by person 1, 2...... respectively(km): ')
    print(travelled)


# Answer obtained:
# [7699.8663420249695, 1017.8146915437312, 4455.859196495479, 17427.97167529595, 11661.24914219831, 6081.879678087437, 1187.0346063844102, 1577.048058278644, 739.685786909313, 540.6760588941578, 62789.27964917755, 1135.3218374429282, 6705.65827116673, 3169.5699418397967, 7172.234009858704, 1491.611097554425, 1065.54192539196, 22328.26996210618, 2263.7041523359135, 1235.506745467672, 750.5716303020857, 2216.918308165503, 6777.587361533187, 7005.122333255826, 8294.881487560595, 52247.4618745825, 2090.9986783034187, 810.5847939601784, 3006.9123576972593, 1087.3921777258977, 7494.756024069514, 251.77146261835094, 495.35468156383035, 707.3376537388967, 3684.461117373416, 1724.6713281970963, 4325.628649473404, 6947.982762231946, 3978.663953263855, 8967.03770244244, 670.7168697875743, 22006.247852299697, 8631.883668954633, 1204.8211415381786, 1755.5440958605955, 1119.528901306788, 783.0168039747856, 265.0832609506473, 506.47053619101376, 226.72369984596835, 4288.853945770225, 2984.634230196266, 13800.818586790376, 618.7842422069826, 73.29820092863979, 1498.5777903941168, 180.08757246453865, 356.6585857897043, 427.3622074810708, 135.67611615629113, 1.4470598198038713, 855.9986228218886, 592472.889299254, 2460.6253510777246, 599.1729392903088, 2167.60503961893, 3945.9325297784044, 15977.998835292077, 10656.50584380035, 751.4347530959062, 595.2110337857663, 2876.0421346939547, 16.914189052903403, 982.1775817333532, 4406.546711396092, 1438.7517328312883, 411.1301837655699, 97.59332734061167, 1173.9354232361031, 576.3451847614543, 102.20326497393748, 490.9420934065672, 23640.82951302542, 2506.1484430111877, 37201.14771564142, 9067.379106383298, 190.4575270397575, 10.201161471486897, 327.7691647286511, 4900.009376907539, 89.42467819567895, 5120.135579523763, 3839.743002274466, 98.66330960357311, 356.6585857897043, 617.7128037833292, 1513.2289325495278, 98.319281477967, 130.5265753334419, 5086.10086427167, 32.05417807762365, 3533.093740813152, 240.13286635436083, 842.6062078911197, 2581.4114267761856, 326.7342921397853, 666.4036796296842, 20.66544211613705, 326.36650291222657, 75.94205224554848, 476.4228593230625, 8772.203452914764, 3562.627472101638, 625.6054570369781, 511.1338317322959, 13642.461532395655, 13.414419971926602, 85.4598845400159, 513.1834972677545, 1418.7255627589493, 501.1457494172194, 205.0634104114295, 4780.403386509127, 1082.6935448092452, 7763.564165805547, 4711.292125292153, 6283.192637886404, 2848.272972693335, 113184.04142721837, 707.7388353509564, 468.4742857238073, 1009.2756513665539, 395.17806280351823, 3522.4045283731803, 4731.785115211515, 2067.785598236771, 157.22138902154427, 1662.6290529001067, 209.2222994205204, 2212.2662135093515, 58130.6901001443, 596.9698848761986, 44150.957900291905, 18.298602818547103, 62747.133677562895, 86.5028499371791, 45.08049757306046, 519.8205085182882, 2355.5753006572168, 345.51208389434146, 356.6585857897043, 17.87120609688419, 81.57176258914748, 137112.09486846998, 303.42727300990043, 986.7754288842947, 205.66074632541523, 854.5534394424211, 4382.325695583823, 2619.401258910459, 254.78171222731373, 110.30809184640245, 595.2164781542663, 168611.22315082926, 56.94169095024818, 273.7640580481123, 48.828499004875106, 6650.513378612053, 9337.064142839157, 779.6356018977226, 102.44663649937971, 28.34884126456664, 23005.686346524035, 65.74781739559664, 3907.3487487691823, 312.4733137040974, 1967.4013351897681, 28.428064913184247, 0.9716821841525018, 1656.1348756159168, 4261.722709452957, 3253.5422234429298]


# Part 2
## Filtering Data for $Bejing$ $City$

In [ ]:
# Filter data Bejing City
# Extrema of Bejing City 
lat_min = 39.72
lat_max = 41.03
lon_min = 115.42
lon_max = 117.50


data = data.loc[(data['latitude'] >= lat_min) & (data['latitude'] <=lat_max) & (data['longitude'] >=lon_min) & (data['longitude'] <= lon_max)]
data.reset_index(drop = True ,inplace=True)
print('Data size for Bejing City:' , len(data), 'rows')


#### Convert `data['time']` from `str` type to `Timestamp` type

In [ ]:

# This parsing of `times`` takes a lot of time 
# Please try uncommenting the # nrows = 100000 and remove the extra ')'
# It will run in a few seconds and gives a resonable result
# I downloaded all 12 maps for the entire data in over 2 hrs 

# There are some faulty datevalues
def to_timestamp(row):
    try:
        return pd.to_datetime(row['time'], format= '%H:%M:%S')
    except:
        print('caught faulty time')
        return pd.NaT
    


data.loc[:,  'time'] = data.apply(lambda row: pd.to_datetime(row['time'], format= '%H:%M:%S'), axis=1)
print('Removed rows with falulty time')
data.loc[:,:].dropna(inplace= True)
data.reset_index(drop=True, inplace=True)



#### Group the `data` into 2 hour slots. Now plot a heat graph for each time slot separately 

In [ ]:
hourly_groups = data.groupby(pd.Grouper(key='time', freq='2H')) 

In [ ]:

def make_heat_map(tup):
    (id, hrly_df) = tup
    beijing_map = folium.Map(location=[39.9042, 116.4074], zoom_start=11)
    locations = hrly_df[['latitude', 'longitude']].values.tolist()

    grad = {0.4: 'blue', 0.6: 'lime', 0.8: 'yellow', 1.0: 'red'}
    heat_layer = HeatMap(locations, name= id, min_opacity=0.3,gradient = grad)
    heat_layer.add_to(beijing_map)
    filename = 'beijing_map' + str(id) + '.html'
    beijing_map.save(filename)

if __name__ == '__main__':
    pool = multiprocess.Pool()
    travelled  = pool.map(make_heat_map, hourly_groups)
    pool.close()
    pool.join()
